In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [ ]:
!pip install -qU "langchain[google-genai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 6.2 MB/s eta 0:00:00


In [39]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 28.7 MB/s eta 0:00:00


In [ ]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

In [46]:
import getpass
import os

# if not os.environ.get("GOOGLE_API_KEY"):
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

Enter API key for Google Gemini: ··········


In [ ]:
loader = PyPDFLoader("/content/TY_syllabus.pdf")
documents = loader.load()

In [48]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(documents)

In [49]:
# 3. Create embeddings & vectorstore
# Index chunks
# _ = vector_store.add_documents(documents=all_splits)

from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(all_splits, embeddings)


In [42]:
# 4. Create retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [50]:
# 6. Create RetrievalQA pipeline
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [51]:
# 7. Simple chat loop
while True:
    query = input("\n📚 Ask about your syllabus: ")
    if query.lower() in ["exit", "quit"]:
        break
    response = qa_chain({"query": query})
    print("\n🤖 Bot:", response["result"])
    print("📄 Sources:", [doc.metadata.get("source") for doc in response["source_documents"]])



📚 Ask about your syllabus: What is the syllabus of Artificial Inteliigence?



🤖 Bot: The syllabus for "CA31231: ARTIFICIAL INTELLIGENCE" for the Third Year B. Tech. Computer Science and Engineering (Artificial Intelligence) 2023 Pattern at Vishwakarma Institute of Information Technology, Pune-48 includes:

*   **Course Code:** CA31231
*   **Course Name:** ARTIFICIAL INTELLIGENCE
*   **Teaching Scheme:**
    *   Credits: 03
    *   Lecture (L): 02 hrs / week
    *   Practical (P): 02 hr / week
*   **Examination Scheme:**
    *   CIE: 20
    *   SCE: 20
    *   ESE: 40
    *   PR: 20
    *   Total: 100
*   **Prerequisites:** Linear Algebra, Probability and Statistics, Discrete Mathematics
*   **Course Objectives:**
    1.  To understand the various characteristics of intelligent agents and advancements of AI.
    2.  To learn the different search strategies in AI, knowledge representation and reasoning in presence of incomplete and uncertain information.
    3.  To learn how to represent knowledge in solving AI problems and to introduce the concepts of... (the re


📚 Ask about your syllabus: explain all assignments of MLOPS, in short



🤖 Bot: The provided document does not explicitly list specific assignments for the MLOps course. However, based on the **Course Outcomes**, students will be expected to perform tasks related to:

1.  **Explaining ML vs. Software Development Lifecycles:** Understanding the fundamental differences.
2.  **Applying MLOps Tools:** Using tools like MLflow, DVC, and BentoML in practical workflows.
3.  **Managing Data, Features, and Experiments:** Effectively organizing and tracking these components.
4.  **Automating ML Pipelines:** Integrating CI/CD for automated machine learning processes.
5.  **Deploying and Monitoring ML Models:** Implementing deployment strategies and monitoring techniques with relevant tools.

These outcomes suggest that assignments would likely involve practical exercises, projects, or lab work demonstrating proficiency in these areas, especially given the "Practical (P): 02 hrs / week" in the teaching scheme and "PR" (Practical) and "TW" (Term Work) components in the 


📚 Ask about your syllabus: exit
